The ResNet convolutional network
==================

In this tutorial I will introduce the ResNet architecture that has been designed by He et al. (2015) in the [article](https://arxiv.org/pdf/1512.03385.pdf) called *"Deep Residual Learning for Image Recognition"*. I will implement the network in tensorflow and I will train it on the CIFAR-10 dataset. ReSNets have been receiving attention in the last few years because they allow the designer to create very deep neural networks. It is important to rememeber that deep networks suffer of many problems, the most important being the [vanishing gradient](https://en.wikipedia.org/wiki/Vanishing_gradient_problem). The vanishing gradient can be intuitively understood thinking about a series of filters. When data pass through those filters a certain amount of information is lost. Deep models have many filters, meaning that there is a gradual deterioration of the data. Resnet solve the problem splitting a deep network into chunks. Each chunk receives the residual output of the previous chunck, plus the unfiltered input that has been passed to the previous chunk.


<p align="center">
<img src="../etc/img/resnet_block.png" width="400">
</p>

The **shortcut** connection introduced by ResNet allows the gradient to pass through during backpropagation, reaching each layer without being diminished. It is important to notice that the trick used in ResNet is to avoid the vanishing gradient instead of really solving it. Veit et al. (2016) pointed out in a [recent article](https://arxiv.org/pdf/1605.06431.pdf) that ResNets use a form of **ensemble by construction**, introducing short paths which can carry gradient throughout the extent of very deep networks. This idea has something in common with the **inception module**, where the layers are stacked in parallel instead than in sequential order. The functioning of a ResNet can be better visualized if we unravel the view of a simple 3-block architecture.

<p align="center">
<img src="../etc/img/resnet_path.png" width="850">
</p>

Each layer of a ResNet has a residual module $f_{i}$ and a skip connection that bypass the module. The residual module is a sequence of convolutions, batch normalization, and ReLu operations. The output of the residual module is computed as follows:

$$ y_{i} = f_{i}(y_{i-1}) + y_{i-1}$$

A description of different variants of shortcuts is given in [He et al. (2016)](https://arxiv.org/pdf/1603.05027.pdf), *"Identity Mappings in Deep Residual Networks"*.


Implementing ReSNet in Tensorflow
--------------------------------------

The implementation in Tensorflow is straightforward and requires just a few modifications to a standard CNN. It is possible to implement different types of ResNets, but the depth is the main factor to take into account. In the original [article](https://arxiv.org/pdf/1512.03385.pdf) of He et al. (2015) five architectures are proposed, spanning from 18 to 152 layers.

<p align="center">
<img src="../etc/img/resnet_type.png" width="750">
</p>

The **convolutional layers** mostly have $3\times3$ filters. When the feature map size is halved, the number of filters is doubled. For instance we have 64 filters in conv2 and 128 in conv3. The downsampling is performed directly by the convolution operations, using a stride of 2.

The **identity** shortcuts (solid line in the picture below) can be directly used when the input and output are of the same dimensions. One of the critical point is the passage between convolutional layers of different size. When the dimensions increase the shortcut still performs identity mapping, with extra zero padding for increasing dimensions (no extra parameters). This particular shortcuts are identified by dotted shortcuts in the image below.

<p align="center">
<img src="../etc/img/resnet_shortcuts.png" width="500">
</p>


In the original work SGD was used to train the network. The dropout was not applied. Batch normalization was used right after each convolution.

Here I will create a ResNet model function that we can easily use into a Tensorflow Estimator framework. For the sake of clarity I will declare each single layer of the network, however it would be better to use a function that can recursively build up the model. The model is quite long but is well organized in scopes. You are invited to compare the different blocks with the one reported in the table below. This is an implementation of the ResNet-18, and the only difference with the original model is that the input matrix has size `[32, 32, 3]`.

In [1]:
import tensorflow as tf

Couldn't import dot_parser, loading of dot files will not be possible.


In [2]:
def my_model_fn(features, labels, mode):
    #Reshape the input    
    x = tf.reshape(features, [-1, 32, 32, 3])
   
    #Feature maps: 32x32@64
    with tf.variable_scope("unit_1"):
        conv1 = tf.layers.conv2d(inputs=x, filters=64, kernel_size=[3, 3], 
                                 padding="same", use_bias=False, activation=None) #no ReLu activation here
        conv1 = tf.layers.batch_normalization(conv1) #apply batch norm
        conv1_out = tf.nn.relu(conv1) #ReLu after the batch norm
    
    #Feature maps: 32x32@64
    with tf.variable_scope("residual_unit_2"):
        #Identity shortcut
        conv2_shortcut = tf.layers.conv2d(inputs=conv1_out, filters=64, kernel_size=[1, 1], 
                                          padding="same", use_bias=False, activation=None)        
        #First convolution of the unit
        conv2_1 = tf.layers.conv2d(inputs=conv1_out, filters=64, kernel_size=[3, 3], 
                                   padding="same", use_bias=False, activation=None)
        conv2_1 = tf.layers.batch_normalization(conv2_1)
        conv2_1 = tf.nn.relu(conv2_1)
        #Second convolution of the unit
        conv2_2 = tf.layers.conv2d(inputs=conv2_1, filters=64, kernel_size=[3, 3], 
                                   padding="same", use_bias=False, activation=None)
        conv2_2 = tf.layers.batch_normalization(conv2_2)
        #Residual is applied
        conv2_2 = conv2_2 + conv2_shortcut
        conv2_2 = tf.nn.relu(conv2_2)
        #Third convolution of the unit
        conv2_3 = tf.layers.conv2d(inputs=conv2_2, filters=64, kernel_size=[3, 3],
                                   padding="same", use_bias=False, activation=None)
        conv2_3 = tf.layers.batch_normalization(conv2_3)
        conv2_3 = tf.nn.relu(conv2_3)
        #Fourth convolution of the unit
        conv2_4 = tf.layers.conv2d(inputs=conv2_3, filters=64, kernel_size=[3, 3],
                                   padding="same", use_bias=False, activation=None)
        conv2_4 = tf.layers.batch_normalization(conv2_4)
        #Residual is applied
        conv2_out = conv2_2 + conv2_4
        conv2_out = tf.nn.relu(conv2_out)

    #Feature maps: 16x16@128
    with tf.variable_scope("residual_unit_3"):
        #Dotted shortcut
        conv3_shortcut = tf.layers.conv2d(inputs=conv2_out, filters=128, kernel_size=[1, 1], strides=2,
                                          padding="same", use_bias=False, activation=None)
        #First convolution of the unit
        conv3_1 = tf.layers.conv2d(inputs=conv2_out, filters=128, kernel_size=[3, 3], strides=2,
                                   padding="same", use_bias=False, activation=None)
        conv3_1 = tf.layers.batch_normalization(conv3_1)
        conv3_1 = tf.nn.relu(conv3_1)
        #Second convolution of the unit
        conv3_2 = tf.layers.conv2d(inputs=conv3_1, filters=128, kernel_size=[3, 3], 
                                   padding="same", use_bias=False, activation=None)
        conv3_2 = tf.layers.batch_normalization(conv3_2)
        #Residual is applied
        conv3_2 = conv3_2 + conv3_shortcut
        conv3_2 = tf.nn.relu(conv3_2)
        #Third convolution of the unit
        conv3_3 = tf.layers.conv2d(inputs=conv3_2, filters=128, kernel_size=[3, 3],
                                   padding="same", use_bias=False, activation=None)
        conv3_3 = tf.layers.batch_normalization(conv3_3)
        conv3_3 = tf.nn.relu(conv3_3)
        #Fourth convolution of the unit
        conv3_4 = tf.layers.conv2d(inputs=conv3_3, filters=128, kernel_size=[3, 3],
                                   padding="same", use_bias=False, activation=None)
        conv3_4 = tf.layers.batch_normalization(conv3_4)
        #Residual is applied
        conv3_out = conv3_2 + conv3_4
        conv3_out = tf.nn.relu(conv3_out)

    #Feature maps: 8x8@256
    with tf.variable_scope("residual_unit_4"):
        #Dotted shortcut
        conv4_shortcut = tf.layers.conv2d(inputs=conv3_out, filters=256, kernel_size=[1, 1], strides=2,
                                          padding="same", use_bias=False, activation=None)
        #First convolution of the unit
        conv4_1 = tf.layers.conv2d(inputs=conv3_out, filters=256, kernel_size=[3, 3], strides=2,
                                   padding="same", use_bias=False, activation=None)
        conv4_1 = tf.layers.batch_normalization(conv4_1)
        conv4_1 = tf.nn.relu(conv4_1)
        #Second convolution of the unit
        conv4_2 = tf.layers.conv2d(inputs=conv4_1, filters=256, kernel_size=[3, 3], 
                                   padding="same", use_bias=False, activation=None)
        conv4_2 = tf.layers.batch_normalization(conv4_2)
        #Residual is applied
        conv4_2 = conv4_2 + conv4_shortcut
        conv4_2 = tf.nn.relu(conv4_2)
        #Third convolution of the unit
        conv4_3 = tf.layers.conv2d(inputs=conv4_2, filters=256, kernel_size=[3, 3],
                                   padding="same", use_bias=False, activation=None)
        conv4_3 = tf.layers.batch_normalization(conv4_3)
        conv4_3 = tf.nn.relu(conv4_3)
        #Fourth convolution of the unit
        conv4_4 = tf.layers.conv2d(inputs=conv4_3, filters=256, kernel_size=[3, 3],
                                   padding="same", use_bias=False, activation=None)
        conv4_4 = tf.layers.batch_normalization(conv4_4)
        #Residual is applied
        conv4_out = conv4_2 + conv4_4
        conv4_out = tf.nn.relu(conv4_out)
        
    #Feature maps: 4x4@512
    with tf.variable_scope("residual_unit_5"):
        #Dotted shortcut
        conv5_shortcut = tf.layers.conv2d(inputs=conv4_out, filters=512, kernel_size=[1, 1], strides=2,
                                          padding="same", use_bias=False, activation=None)
        #First convolution of the unit
        conv5_1 = tf.layers.conv2d(inputs=conv4_out, filters=512, kernel_size=[3, 3], strides=2,
                                   padding="same", use_bias=False, activation=None)
        conv5_1 = tf.layers.batch_normalization(conv5_1)
        conv5_1 = tf.nn.relu(conv5_1)
        #Second convolution of the unit
        conv5_2 = tf.layers.conv2d(inputs=conv5_1, filters=512, kernel_size=[3, 3], 
                                   padding="same", use_bias=False, activation=None)
        conv5_2 = tf.layers.batch_normalization(conv5_2)
        #Residual is applied
        conv5_2 = conv5_2 + conv5_shortcut
        conv5_2 = tf.nn.relu(conv5_2)
        #Third convolution of the unit
        conv5_3 = tf.layers.conv2d(inputs=conv5_2, filters=512, kernel_size=[3, 3],
                                   padding="same", use_bias=False, activation=None)
        conv5_3 = tf.layers.batch_normalization(conv5_3)
        conv5_3 = tf.nn.relu(conv5_3)
        #Fourth convolution of the unit
        conv5_4 = tf.layers.conv2d(inputs=conv5_3, filters=512, kernel_size=[3, 3],
                                   padding="same", use_bias=False, activation=None)
        conv5_4 = tf.layers.batch_normalization(conv5_4)
        #Residual is applied
        conv5_out = conv5_2 + conv5_4
        conv5_out = tf.nn.relu(conv5_out)
        
    #Pooling and output
    with tf.variable_scope("unit_6"):
        pool = tf.layers.average_pooling2d(inputs=conv5_out, pool_size=[2, 2], strides=1,
                                           padding="same")
        pool_flat = tf.reshape(pool, [-1, 4 * 4 * 512])
        logits = tf.layers.dense(inputs=pool_flat, units=10)

    #PREDICT mode
    if mode == tf.estimator.ModeKeys.PREDICT:
        predictions = {"classes": tf.argmax(input=logits, axis=1),
                       "probabilities": tf.nn.softmax(logits, name="softmax_tensor"),
                       "logits": logits}
        return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)
    #TRAIN mode
    elif mode == tf.estimator.ModeKeys.TRAIN:
        loss = tf.losses.softmax_cross_entropy(onehot_labels=labels, logits=logits)
        #optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.001)
        optimizer = tf.train.AdamOptimizer(learning_rate=0.001)
        train_op = optimizer.minimize(loss=loss, global_step=tf.train.get_global_step())
        accuracy = tf.metrics.accuracy(labels=tf.argmax(labels, axis=1), predictions=tf.argmax(logits, axis=1))
        tf.summary.scalar('accuracy', accuracy[1]) #<-- accuracy[1] to grab the value
        #tf.summary.image("input_features", tf.reshape(features, [-1, 32, 32, 3]), max_outputs=4)
        #tf.summary.image("c1_k1_feature_maps", tf.reshape(conv1[:, :, :, 0], [-1, 32, 32, 1]), max_outputs=4)
        logging_hook = tf.train.LoggingTensorHook({"accuracy" : accuracy[1]}, every_n_iter=10)
        return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op, training_hooks =[logging_hook])
    #EVAL mode
    elif mode == tf.estimator.ModeKeys.EVAL:
        loss = tf.losses.softmax_cross_entropy(onehot_labels=labels, logits=logits)
        accuracy = tf.metrics.accuracy(labels=tf.argmax(labels, axis=1), predictions=tf.argmax(logits, axis=1))
        eval_metric = {"accuracy": accuracy}
        return tf.estimator.EstimatorSpec(mode=mode, loss=loss, eval_metric_ops=eval_metric)

Now we can create the etimator object and link our model function to it. We also have to specify a folder where the log will be stored. Pay attention to the path where you are saving the model.

In [3]:
resnet = tf.estimator.Estimator(model_fn=my_model_fn, model_dir="./tf_resnet_model")

Training the model on CIFAR-10
------------------------------------

In order to train the network you need the CIFAR-10 TFRecord files. You can easily produce the TFRecords following [this notebook](../cifar10/cifar10.ipynb) that is part of the TensorBag repository. Once you have the files you have to preprocess them and aggregate a dataset. Everything is done using the function below. Once the dataset is ready it is possible to run the training. On my laptop (Intel i5, 4 cores) the training takes approximately 30 minutes for 500 iterations, and 3.2GB of RAM. Remember that a checkpoint is periodically stored by Tensorflow in the folder specified when you created the estimator object. If you run again the training snippet the checkpoint will be loaded automatically and the training will start from the last point.

In [ ]:
def my_input_fn():
    def _parse_function(example_proto):
        features = {"image": tf.FixedLenFeature((), tf.string, default_value=""),
                    "label": tf.FixedLenFeature((), tf.int64, default_value=0)}
        parsed_features = tf.parse_single_example(example_proto, features)
        image_decoded = tf.decode_raw(parsed_features["image"], tf.uint8) #char -> uint8
        image_reshaped = tf.reshape(image_decoded, [32, 32, 3])
        image = tf.cast(image_reshaped, tf.float32)
        label_one_hot = tf.one_hot(parsed_features["label"], depth=10, dtype=tf.int32)
        return image, label_one_hot

    tf_train_dataset = tf.data.TFRecordDataset("./cifar10_train.tfrecord")
    tf_train_dataset = tf_train_dataset.map(_parse_function)
    tf_train_dataset.cache() # caches entire dataset
    tf_train_dataset = tf_train_dataset.shuffle(buffer_size = 50000 * 2)
    tf_train_dataset = tf_train_dataset.repeat(11)
    tf_train_dataset = tf_train_dataset.batch(32)
    print "Train dataset: " + str(tf_train_dataset)
    
    iterator = tf_train_dataset.make_one_shot_iterator()
    batch_features, batch_labels = iterator.get_next()
    return batch_features, batch_labels

In [ ]:
tf.logging.set_verbosity(tf.logging.INFO)

In [ ]:
resnet.train(input_fn=my_input_fn, steps=4000)

Tesing the model
-------------------

In order to test the model we have to define an input function to parse the test set. Depending on the computational capability of your machine you may want to speed upt the test. You can set `steps` parameter in the `evaluate()` method to a value of 100 or 1000, meaning that you consider only part of the training set.

In [4]:
def my_eval_input_fn():
    def _parse_function(example_proto):
        features = {"image": tf.FixedLenFeature((), tf.string, default_value=""),
                    "label": tf.FixedLenFeature((), tf.int64, default_value=0)}
        parsed_features = tf.parse_single_example(example_proto, features)
        image_decoded = tf.decode_raw(parsed_features["image"], tf.uint8) #char -> uint8
        image_reshaped = tf.reshape(image_decoded, [32, 32, 3])
        image = tf.cast(image_reshaped, tf.float32)
        label_one_hot = tf.one_hot(parsed_features["label"], depth=10, dtype=tf.int32)
        return image, label_one_hot

    tf_test_dataset = tf.data.TFRecordDataset("./cifar10_test.tfrecord")
    tf_test_dataset = tf_test_dataset.map(_parse_function)
    tf_test_dataset.cache() # caches entire dataset
    tf_test_dataset = tf_test_dataset.shuffle(buffer_size = 10000 * 2)
    tf_test_dataset = tf_test_dataset.repeat(1)
    tf_test_dataset = tf_test_dataset.batch(1)
    print "Test dataset: " + str(tf_test_dataset)
    
    iterator = tf_test_dataset.make_one_shot_iterator()
    batch_features, batch_labels = iterator.get_next()
    return batch_features, batch_labels

In [8]:
resnet.evaluate(input_fn=my_eval_input_fn, steps=1000)

Test dataset: <BatchDataset shapes: ((?, 32, 32, 3), (?, 10)), types: (tf.float32, tf.int32)>


{'accuracy': 0.547, 'global_step': 6000, 'loss': 1.297175}

Conclusions
-------------

In this tutorial we saw how the ResNet-18 CNN works and how it is possible to train it on the CIFAR-10 dataset. You can further improve the performance of the network going deeper. If you have enough patience (or a good GPU) you can try to implement a ResNet-34 or a ResNet-50.

**Copyright (c)** 2018 Massimiliano Patacchiola, MIT License